In [9]:
print("vi")

vi


In [89]:
import os
import psycopg2
from psycopg2 import sql

# Set the PGDATABASE environment variable
os.environ["PGDATABASE"] = "ernieplus"

def insert_values_into_table(values_list):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect("")
        cur = conn.cursor()

        
        args = ','.join(cur.mogrify("(%s,%s,%s, %s, %s, %s, %s, %s)", i).decode('utf-8')
                for i in values_list)
        
        cur.execute("INSERT INTO hm31.pubmed_all_xmls VALUES " + (args))

        conn.commit()
        print("Inserted values successfully")

    except Exception as e:
        print("Error:", e)

    finally:
        cur.close()
        conn.close()

# Example usage:
values_to_insert = [
    (123425, '', '', '', 0, 0, '', 'doi1'),
    (543251, '5678-1234', 'Grant3, Grant4', 'Chemical3, Chemical4', 2021, 2020, 'Term3, Term4','doi2')
]

insert_values_into_table(values_to_insert)


Inserted values successfully


In [11]:
import xmltodict

def parse(file_path):
    try:
        # Open and read the XML file
        with open(file_path, 'r', encoding='utf-8') as file:
            xml_contents = file.read()
        print("red")
        # Parse the XML content using xmltodict
        xml_dict = xmltodict.parse(xml_contents)
        print("parsed")
        # Print the parsed XML as a Python dictionary
    except Exception as e:
        print("Error:", e)
        
    return xml_dict

xml_dict = parse('/shared/hm31/xml_data/pubmed23n0001.xml')



red
parsed


In [63]:
article_id_list = temp['PubmedData']['ArticleIdList']['ArticleId']
print('doi' in str(article_id_list[1]))
# print(article_id_list[1])
print(article_id_list[1].keys())
print(article_id_list[1]['#text'])


# Iterate through the list of ArticleId entries
# for article_id in article_id_list['ArticleId']:
#     if article_id.get('@IdType') == 'doi':
#         doi = article_id['#text']
#         print("DOI:", doi)
#         break

True
odict_keys(['@IdType', '#text'])
10.1016/0006-2944(75)90147-7


In [25]:


text = """
<Day>16</Day>
</PubMedPubDate>
<PubMedPubDate PubStatus="medline">
<Year>1976</Year>
<Month>3</Month>
<Day>16</Day>
<Hour>0</Hour>
<Minute>1</Minute>
</PubMedPubDate>
<PubMedPubDate PubStatus="entrez">
<Year>1976</Year>
<Month>3</Month>
<Day>16</Day>
<Hour>0</Hour>
<Minute>0</Minute>
</PubMedPubDate>
</History>
<PublicationStatus>ppublish</PublicationStatus>
<ArticleIdList>
<ArticleId IdType="pubmed">4311</ArticleId>
<ArticleId IdType="doi">10.1111/j.1432-1033.1976.tb10218.x</ArticleId>
</ArticleIdList>
</PubmedData>
</PubmedArticle>
</PubmedArticleSet>
"""

doi_pattern = r'\b10\.\d{4,}/\S+'
doi_pattern = r'\b10\.\d{4,}(?:\.\d+)*\/[^\s]+'


# Find all DOIs in the text
dois = re.findall(doi_pattern, text)

# Print the found DOIs
for doi in dois:
    print("Found DOI:", doi)

Found DOI: 10.1111/j.1432-1033.1976.tb10218.x</ArticleId>


In [32]:
import re
def find_doi(string):
    doi_pattern = r'\b10\.\d{4,}(?:\.\d+)*\/[^\s]+'
    dois = re.findall(doi_pattern, string)
    return dois

    

In [81]:
from tqdm import tqdm

def parse_single_record(xml_dict, originial_dict):
    mesh_headings = []
    grants = []
    year = ""
    journal_ISSN = ""
    chemical_list = []
    meta_data = {}
    pub_year = ""
    PMID = ""
    doi = ""


    try:
        xml_dict = dict(xml_dict)
        
        
#         print(xml_dict)
        
        try:
            if 'PMID' in xml_dict:
                PMID = xml_dict['PMID']['#text']
        except:
            pass
        
        try:
            if 'DateCompleted' in xml_dict:
                new_dic = dict(xml_dict['DateCompleted'])
                if 'Year' in new_dic:
                    year = new_dic['Year']

            else:
                  pass
    
        except:
            pass

        
        try:
            if 'Article' in xml_dict:
                new_dic = dict(xml_dict['Article'])
                journal_ISSN = new_dic['Journal']['ISSN']['#text']
                
        except:
            pass
    
        try:
            if 'Article' in xml_dict:
                new_dic = dict(xml_dict['Article'])
                pub_year = new_dic['Journal']['JournalIssue']['PubDate']['Year']

        except:
            pass
        
        
        try:

                article_id_list = originial_dict['PubmedData']['ArticleIdList']['ArticleId']
                
                for element in article_id_list:
                    if 'doi' in str(element).lower():
                        doi = element['#text']
                        break
#                 print('doi' in str(article_id_list[1]))
#                 # print(article_id_list[1])
#                 print(article_id_list[1].keys())
#                 print(article_id_list[1]['#text'])
#                 #print('doi' in str(article_id_list[1]))

        except:
            pass
     
        

        try:
            if 'Article' in xml_dict: 
                new_dic = dict(xml_dict['Article'])
                
            if 'Title' in  new_dic['Journal'].keys():
                journal_title = new_dic['Journal']['Title']
        
        except:
            pass

        try:
            if 'Article' in xml_dict: 
                new_dic = dict(xml_dict['Article'])

            if 'GrantList' in new_dic:
                if type(new_dic['GrantList']['Grant']) == list:
                    for grant in new_dic['GrantList']['Grant']:
                        if 'GrantID' in grant:
                            grants.append((grant['GrantID']))
                            
                else:
                     grants.append(new_dic['GrantList']['Grant']['GrantID'])
                     pass   
        except:
            pass
        
        
        try:
            if 'MeshHeadingList' in xml_dict:  
                if type(xml_dict['MeshHeadingList']['MeshHeading']) == list:
                    for mesh in xml_dict['MeshHeadingList']['MeshHeading']:
                        if '@Type' in mesh['DescriptorName'].keys() and mesh['DescriptorName']['@Type'] == 'Geographic':
                            continue
                
                
                        mesh_headings.append((mesh['DescriptorName']['@UI']))
            
                else:
                    
                    mesh = xml_dict['MeshHeadingList']['MeshHeading']['DescriptorName']
                    if  (not '@Type' in mesh.keys() or mesh['@Type'] != 'Geographic'):
                        mesh_headings.append(xml_dict['MeshHeadingList']['MeshHeading']['DescriptorName']['@UI'])
                    
        except:
            pass
        
        
        try:
        
            if 'ChemicalList' in xml_dict:
               if type(xml_dict['ChemicalList']['Chemical']) == list:
                   for substance in xml_dict['ChemicalList']['Chemical']:
                        if substance['NameOfSubstance']['@UI'][0].lower() == 'c':
                           chemical_list.append(substance['NameOfSubstance']['@UI'])
                        
               else:
                    
                    if xml_dict['ChemicalList']['Chemical']['NameOfSubstance']['@UI'][0].lower() == 'c':
                           chemical_list.append(xml_dict['ChemicalList']['Chemical']['NameOfSubstance']['@UI'])

                            
        except Exception as e:
            print(e)
            print("ERR")
            pass
        
    except:
        pass


    if len(year) == 0:
        year = pub_year
        
    meta_data = {'PMID': PMID, 'mesh': mesh_headings, 'grants': grants, 'year': year, 
                 'journal_ISSN': journal_ISSN,'journal_title': journal_title,
                 'chemical' : chemical_list, 'pub_year': pub_year, 'doi': doi} 
    
    return meta_data

meta_data_array = []

for i in tqdm(range(len(xml_dict['PubmedArticleSet']['PubmedArticle']))):
    rec = xml_dict['PubmedArticleSet']['PubmedArticle'][i]['MedlineCitation']
    total = xml_dict['PubmedArticleSet']['PubmedArticle'][i]
    x=parse_single_record(rec, total)
    meta_data_array.append(x)

    
    

100%|██████████| 30000/30000 [00:14<00:00, 2050.99it/s]


In [84]:
#Convert dict to query
#meta_data = {'mesh': mesh_headings, 'grants': grants, 'year': year, 
#                 'journal_ISSN': journal_ISSN,'journal_title': journal_title,
#                 'chemical' : chemical_list, 'pub_year': pub_year} 

#(543251, '5678-1234', 'Grant3, Grant4', 'Chemical3, Chemical4', 2021, 2020, 'Term3, Term4')
def convert_dict_to_query(dic):
    if dic['year'] == '':
        year = 0
    
    else:
        year = dic['year']
        
    
    if dic['pub_year'] == '':
        pub_year = 0
    
    else:
        pub_year = dic['pub_year']
        
        
    query = (int(dic['PMID']), dic['journal_ISSN'], ' '.join(dic['grants']), ' '.join(dic['chemical']), 
            pub_year, year, ' '.join(dic['mesh']), dic['doi'] )
    
    return query

In [96]:
query_array = []

for i in meta_data_array:
    query_array.append(convert_dict_to_query(i))

num = 0
for idx, query in enumerate(query_array):
    if len(query[-1]) > 0:
        num += 1
    
    if idx < 5:
        print(query)

print(num)
        
print(query_array[0:5])
insert_values_into_table(query_array)

(1, '0006-2944', 'F32 AG064886 MC_UU_12013/5', '', '1975', '1976', 'D000445 D000818 D001826 D002245 D005561 D000882 D006801 D006863 D007700 D000432 D008722 D011549', '10.1016/0006-2944(75)90147-7')
(2, '1090-2104', '', '', '1975', '1976', 'D005583 D009682 D008958 D008968 D009243 D009249 D013329 D013696', '10.1016/0006-291x(75)90482-9')
(4, '1090-2104', 'BB/C008219/1 G1100377 G1100377', '', '1975', '1976', 'D001692 D002478 D002553 D002738 D003911 D005347 D005966 D006801 D007966 D008247 D010873 D012867 D013429', '10.1016/0006-291x(75)90506-9')
(3, '0006-291X', '', '', '1975', '1976', 'D000818 D001665 D002104 D002256 D002417 D006801 D006863 D009682 D008628 D011485 D011487 D015032', '10.1016/0006-291x(75)90498-2')
(5, '1090-2104', '', '', '1975', '1976', 'D000818 D003063 D003201 D006422 D008667 D008958 D009124 D011487 D013045', '10.1016/0006-291x(75)90508-2')
16743
[(1, '0006-2944', 'F32 AG064886 MC_UU_12013/5', '', '1975', '1976', 'D000445 D000818 D001826 D002245 D005561 D000882 D006801 D

In [100]:
def parallelize(file_path):
    xml_dict = parse(file_path)
    
    meta_data_array = []

    for i in range(len(xml_dict['PubmedArticleSet']['PubmedArticle'])):
        rec = xml_dict['PubmedArticleSet']['PubmedArticle'][i]['MedlineCitation']
        total = xml_dict['PubmedArticleSet']['PubmedArticle'][i]
        x=parse_single_record(rec, total)
        meta_data_array.append(x)
        
    query_array = []

    for i in meta_data_array:
        query_array.append(convert_dict_to_query(i))

    
    insert_values_into_table(query_array)


In [101]:
import multiprocessing

nest_dir = '/shared/hm31/xml_data/'
files = os.listdir(nest_dir)

lst = []

for file in files:
    lst.append((nest_dir+file,))
    
with multiprocessing.Pool(processes=80) as pool:
    results = pool.starmap(parallelize, lst)


red
red
red
red
parsed
Inserted values successfully
parsed
parsed
Inserted values successfully
Inserted values successfully
parsed
Inserted values successfully
red
red
red
red
parsed
red
Inserted values successfully
red
parsed
Inserted values successfully
parsed
parsed
Inserted values successfully
Inserted values successfully
red
red
red
red
red
parsed
red
Inserted values successfully
red
parsed
Inserted values successfully
red
red
parsed
Inserted values successfully
parsed
Inserted values successfully
parsed
red
parsed
Inserted values successfully
red
Inserted values successfully
parsed
Inserted values successfully
red
parsed
Inserted values successfully
parsed
Inserted values successfully
red
parsed
red
Inserted values successfully
red
parsed
Inserted values successfully
red
red
red
parsed
red
Inserted values successfully
parsed
red
Inserted values successfully
red
red
red
red
red
parsed
Inserted values successfully
parsed
red
red
red
Inserted values successfully
red
red
red
red
red


Inserted values successfully
Inserted values successfully
parsed
red
red
parsed
Inserted values successfully
red
parsed
Inserted values successfully
red
parsed
Inserted values successfully
Inserted values successfully
red
Inserted values successfully
parsed
Inserted values successfully
parsed
Inserted values successfully
parsed
Inserted values successfully
Inserted values successfully
parsed
parsed
parsed
red
red
Inserted values successfully
parsed
red
red
Inserted values successfully
parsed
parsed
Inserted values successfully
Inserted values successfully
Inserted values successfully
red
Inserted values successfully
Inserted values successfully
red
Inserted values successfully
parsed
parsed
parsed
parsed
Inserted values successfully
red
parsed
Inserted values successfully
Inserted values successfully
red
red
Inserted values successfully
Inserted values successfully
parsed
red
parsed
red
parsed
red
parsed
parsed
red
Inserted values successfully
red
Inserted values successfully
parsed
In

Inserted values successfully
red
red
red
red
red
parsed
red
red
red
parsed
red
parsed
parsed
red
red
parsed
parsed
parsed
parsed
parsed
red
red
parsed
parsed
parsed
parsed
red
red
red
parsed
red
parsed
red
red
parsed
red
parsed
parsed
parsed
parsed
parsed
redred

parsed
parsed
parsed
Inserted values successfullyInserted values successfully
Inserted values successfully

parsed
parsed
Inserted values successfully
red
parsed
Inserted values successfully
parsed
Inserted values successfully
red
Inserted values successfully
Inserted values successfullyInserted values successfully

parsed
parsed
red
parsed
red
red
Inserted values successfullyInserted values successfullyInserted values successfully


Inserted values successfully
Inserted values successfully
parsed
red
Inserted values successfully
Inserted values successfully
Inserted values successfully
Inserted values successfully
Inserted values successfully
red
red
parsed
Inserted values successfully
Inserted values successfully
red
red
Ins

parsed
red
Inserted values successfully
parsed
parsed
Inserted values successfully
Inserted values successfully
red
Inserted values successfully
Inserted values successfully
red
parsed
parsed
parsed
Inserted values successfully
Inserted values successfully
red
parsed
red
red
red
Inserted values successfully
Inserted values successfully
Inserted values successfully
red
red
parsed
red
red
parsed
red
red
red
red
parsed
red
red
parsed
parsed
parsed
parsed
parsed
parsed
parsed
parsed
parsed
parsed
parsed
parsed
parsed
Inserted values successfully
red
red
red
parsed
parsed
red
Inserted values successfully
parsed
Inserted values successfully
Inserted values successfully
Inserted values successfullyInserted values successfully

Inserted values successfully
Inserted values successfully
Inserted values successfully
parsed
Inserted values successfully
Inserted values successfully
Inserted values successfully
Inserted values successfully
red
parsed
parsed
Inserted values successfully
parsed
Insert

Inserted values successfully
parsed
Inserted values successfully
Inserted values successfully
Inserted values successfully
red
parsed
Inserted values successfully
Inserted values successfully
parsed
parsed
red
Inserted values successfully
Inserted values successfully
Inserted values successfully
parsed
Inserted values successfully
red
Inserted values successfullyInserted values successfully

parsed
Inserted values successfully
Inserted values successfully
Inserted values successfully
parsed
Inserted values successfully
Inserted values successfully
Inserted values successfully
parsed
Inserted values successfully
Inserted values successfully
Inserted values successfully
Inserted values successfully
red
parsedred

Inserted values successfully
Inserted values successfully
Inserted values successfully
parsed
red
Inserted values successfully
Inserted values successfully
Inserted values successfully
red
parsed
parsed
parsed
red
red
Inserted values successfully
red
parsed
Inserted values succe

parsed
Inserted values successfully
Inserted values successfully
parsed
Inserted values successfully
red
red
red
parsed
Inserted values successfully
parsed
parsed
red
Inserted values successfully
parsed
parsed
parsed
Inserted values successfully
Inserted values successfully
Inserted values successfully
red
Inserted values successfully
Inserted values successfully
parsed
red
red
parsed
Inserted values successfully
red
parsed
Inserted values successfully
Inserted values successfully
red
parsed
Inserted values successfully
Inserted values successfully
parsed
parsed
parsed
Inserted values successfully
Inserted values successfully
parsed
parsed
red
parsed
Inserted values successfully
Inserted values successfully
Inserted values successfully
red
red
red
parsed
parsed
parsed
parsed
Inserted values successfully
Inserted values successfully
parsed
Inserted values successfully
Inserted values successfully
Inserted values successfully
red
Inserted values successfully
red
parsed
parsed
red
parsed


TypeError: sequence item 0: expected str instance, NoneType found